In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #读取本地.env文件，里面定义了OPENAI_API_KEY和代理配置

In [3]:
from openai import OpenAI
import json

def get_completion(messages, model="gpt-3.5-turbo"):
    client = OpenAI()
    chat_completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        functions=[{
                       "name": "sum",
                       "description": "计算数组中所有数字的和",
                       "parameters": {
                           "type": "object",
                           "properties": {
                               "numbers": {
                                   "type": "array",
                                   "items": {
                                       "type": "number"
                                   }
                               }
                           }
                       }
                   }]
    )
    return chat_completion.choices[0].message

In [4]:
prompt = "Tell me the sum of 1,2,3,4,5."
messages = [
    {"role":"system","content":"你是一个小学数学老师，你要教学生加法"},
    {"role":"user","content":prompt}
]
response = get_completion(messages)
messages.append(response)#把大模型的回复加到对话中
print(response)

if(response.function_call):
    if(response.function_call.name) == "sum":
        args = json.loads(response.function_call.arguments)
        result = sum(args["numbers"])
        print(result)
        messages.append(
            {"role": "function","name":"sum", "content":str(result)})
        print(get_completion(messages).content)


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"numbers":[1,2,3,4,5]}', name='sum'), tool_calls=None)
15
The sum of 1, 2, 3, 4, and 5 is 15.
